# **Defined: What is photoplethysmography?**

Photoplethysmography (PPG) is a simple, low-cost, and non-invasive optical technique that can be used to detect blood volume changes in a microvascular bed of tissue at the skin surface.  It is widely used in smart watches and is easily recognized as the “green light” at the back of the fitbit watch.  There has been research to simulate rPPG by measuring changes in red, green, and blue light reflection from the skin, quantifying the contrast between specular reflection and diffused reflection. Specular reflection is the pure light reflection from the skin. Diffused reflection is the reflection that remains following the absorption and scattering of light in skin. Many protocols exist to simulate this “colored light” effect, and while some are more accurate than others there is a plethora of cases in which these protocols can be used, one use case in particular is telemedicine.  




# Application: Photoplethysmography is the future of mobile health monitoring

As noted earlier, there are a plethora of cases in which photoplethysmography can be used, some of the most prominant use cases are in telemedicine. 
One use case of photoplethysmography is the measurement of blood pressure (BP)*(Elgendi, et al)*. High blood pressure is associated with many chronic disease conditions. Being able to accurately and frequently measure BP outside of a clinical setting, using mobile or wearable devices is a breakthrough for medical technology. Recent advances in technology have brought us one step closer to using Remote photoplethysmography (RPPG) to measure BP on mobile devices. The protocol we work with in this project does not measure BP, but instead measures heart rate. Future work would take the learnings from this project and build on top of it to make a mobile phone RPPG protocol for BP measurement.

# Understanding: How photoplethysmography works

PPG sensors rely on reflecting light off of the skin and observing the response on a photo-detector, or by observing the skin’s absorption of light. For our case, we transmit light at a wavelength that our skin is good at reflecting and then measure this reflected light.

Overall the process is as such:

1. The heart pumps blood through the body.
2. As the volume of blood changes, the way the skin absorbs and reflects light changes.
3. By measuring the change in light absorption, we can extract a measure of how quickly the volume of blood is changing and, thus, extract our heart-rate.

<img src="photopletis_work.png">

# Interesting Note: When using fingers and toes use red, otherwise green works(in theory).

I orginally thought from reading literature(see sources at the bottom) that using the green signal from the recorded phone samples would provide better data. This was actually erranous. When it comes to using RPPG for measuring Heart rate using the arm and wrist, green signal can be pretty effective. When it comes to using the finger tip however, it is best to isolate the red signal and extract the heart rate from there. I project this is because the finger tips do not have the same type of vein structure as the vessels in our arms. When you look at the vessels on a person's arms -- you can sometimes see the blue / green color of the veins when a person is actively pumping alot of blood (think after running or exerting energy). However, as for the hands, they turn red. This phenomenon lends it self to better RPPG accuracy when the protol uses red. So for our project below, we use red signal to isolate heart rate. 

# Prep: Taking your own heart rate sample

Before we can extract the heart-rate, we must record it.

1. Open your camera app on your phone and prepare to record video (don’t start the recording yet, though).
2. Turn on the flash.
3. Place your finger such that it covers both the flash AND the camera, like the image below. Note that you need to adjust this based on your flash/camera layout. Just make sure your finger covers both.
4. Record a video of at least 20 seconds.
5. Save video to computer/ Google Drive to be accesible to this script

<img src="ppg_use_pic.jpg">

In [5]:
import cv2
import numpy as np
from scipy import signal
import os
import shutil
"""
# This Function Works as follows:
1. Takes input as the path of the finger video 
2. Returns the resting heartrate in bpm as a number 


- It first gets frames from video at rate 20 frames per second
- Extract channels from the images 
- then it computes the average of the red channels and saves them in pixels_averages
- Then do a HPF to the average
- Truncate wonky data from  beginning of series
- Take FFT to the filtered data and the compute the heartrate in bpm.
"""
overall_results = []

def HeartRateFinger(Video_Path):
    os.makedirs("frames")  #make a directory called frames to save the images in
    vidcap = cv2.VideoCapture(Video_Path)  #the directory path to the video of interest
    images = []

    #take in images and get indidvual frames
    def getFrame(sec):
        vidcap.set(cv2.CAP_PROP_POS_MSEC, sec * 1000)  #VideoCaptureProperties to capture 
        hasFrames, image = vidcap.read()
        if hasFrames:
            cv2.imwrite("frames/image" + str(count) + ".jpg", image)  # save frame as JPG file
        images.append("frames/image" + str(count) + ".jpg")
        return hasFrames

    #set up frame rate
    sec = 0
    frameRate = 0.05  #capture frame each 0.05 seconds --> (1 second) / .05 = 20 seconds 20 fps
    count = 1
    success = getFrame(sec)  #success tracker
    while (success):
        count = count + 1
        sec = sec + frameRate
        sec = round(sec, 2)
        success = getFrame(sec)

    #Get the channels from images and Find the average of RED channel for each frame
    pixels_averages = []
    for i in range(len(images) - 1):
        bgr_image = cv2.imread(images[i])
        blue_channel, green_channel, red_channel = cv2.split(bgr_image)  #splits into the 3 color channels (RGB)
        average = np.mean(red_channel)  #mean of all the red channels  is the sum all pixels in channel and divide by number of pixels
        pixels_averages.append(average)  #add this average to all the other frames array
    pixels_averages = np.divide(np.array(pixels_averages), 255)  #normalize our averages between 0-1

    #Clean and filter data (butterworth filter)
    sos = signal.butter(4, 1, 'hp', fs=20, output='sos')  # setting up signal.butter(kind of frequency,framerate, name of method)
    filtered = signal.sosfilt(sos, pixels_averages) #Applying filter

    # truncate wonky data from  beginning of series
    filtered = filtered[40:]

    #Perform FFT to find frequency of max amplitude
    Sample_rate = 20  #same as the frame rate is the sampling rate
    BW = Sample_rate / 2  #bandwidth (range of frequency) in signal processing
    fft = np.absolute(np.fft.fft(filtered))
    frames_len = len(filtered)
    frequancies = np.arange(0, BW, Sample_rate / frames_len)
    fft = fft[0:len(frequancies)]

    #Convert back to BPM from hz
    heartrate = np.round(frequancies[np.argmax(fft)] * 60)  #take the highest peak frequency and multiply by 60 and round it
    shutil.rmtree("frames")  #remove folder made to keep the directory clean
    overall_results.append(heartrate)
    return heartrate



If it found frames, then True, else false: False
Your Heart rate is : 77.0 bpm


In [ ]:
#script expected outcome while Resting is 74-78 BPM
path = "95.MOV"
my_heart_rate = HeartRateFinger(path)
print("Your Heart rate is : {} bpm".format(my_heart_rate))

In [6]:
#script expected outcome while Resting is 69-77 BPM
path = "resting1.mp4"
my_heart_rate = HeartRateFinger(path)
print("Your Heart rate is : {} bpm".format(my_heart_rate))

If it found frames, then True, else false: False
Your Heart rate is : 70.0 bpm


In [7]:
#script expected outcome while Resting is 69-77 BPM
path = "RESTING2.mp4"
my_heart_rate = HeartRateFinger(path)
print("Your Heart rate is : {} bpm".format(my_heart_rate))

If it found frames, then True, else false: False
Your Heart rate is : 70.0 bpm


In [ ]:
#script expected outcome after activity was 83-88 BPM (50 jumping jacks)
path = "active1.mp4"
my_heart_rate = HeartRateFinger(path)
print("Your Heart rate is : {} bpm".format(my_heart_rate))

In [11]:
#script expected outcome after activity was 75-85 BPM (jogging for a bit)
path =  "active2.mp4"
my_heart_rate = HeartRateFinger(path)
print("Your Heart rate is : {} bpm".format(my_heart_rate))

If it found frames, then True, else false: False
Your Heart rate is : 75.0 bpm


In [ ]:
#Overall results laid out with their error rate as well
print(overall_results)


# Code Results Discussion

I used a [Zacurate Pro Series 500DL Fingertip Pulse Oximeter](https://www.amazon.com/Zacurate-Fingertip-Oximeter-Saturation-batteries/dp/B00SCPB9KU) to benchmark my results. The code experiences some interesting patterns and possible errors. I list them below. 

1. The code has an error margin of about 7 BPM. So the data seems to range between the expected heartrate plus 7 BPM or minus 7BPM.
2. The data did not notice huge changes in heart rate unless there was extreme exercise IE 50 jumping jacks vs jogging for a bit produced different results
3. I suspect that there may be error related to measuring heart rate using the benchmark Fingertip Pulse Oximeter and the phone camera because I would utilze them at the same time on different hands. So for instance after 50 jumping jacks, I would put the Fingertip Pulse Oximeter on my right index finger and the phone camera mechanism on my left index finger. Though the right and left hand are normally intune, thats not always the case. This could be a source of error.
4. I suspect that there may also be error related to make, model and light intensities from my iphone 11 that would cause variability in results. 

# Sources and further reading

1. Elgendi, M., Fletcher, R., Liang, Y. et al. The use of photoplethysmography for assessing hypertension. npj Digit. Med. 2, 60 (2019). https://doi.org/10.1038/s41746-019-0136-7 
2. https://github.com/prouast/heartbeat
3. https://github.com/qiriro/PPG
4. Sun, Yu, and Thakor, Nitish. “Photoplethysmography Revisited: From Contact to Noncontact, From Point to Imaging.” IEEE Transactions on Biomedical Engineering, vol. 63, no. 3, 2016, pp. 463–477.
5. Sun, Yu, et al. “Noncontact Imaging Photoplethysmography to Effectively Access Pulse Rate Variability.” Journal of Biomedical Optics, vol. 18, no. 6, 2013, p. 061205.
6. McDuff, Daniel, et al. “Non-Contact Imaging of Peripheral Hemodynamics during Cognitive and Psychological Stressors.” Scientific Reports, vol. 10, no. 1, 2020, p. 10884.
7. Maestre-Rendon, J. Rodolfo, et al. “A Non-Contact Photoplethysmography Technique for the Estimation of Heart Rate via Smartphone.” Applied Sciences, vol. 10, no. 1, 2019, p. 154.
